In [ ]:
# В РЕЖИМІ РЕАЛЬНОГО ЧАСУ
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError


%matplotlib tk


CHUNK = 1024 * 2            
FORMAT = pyaudio.paInt16    
CHANNELS = 1                
RATE = 44100              
 
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

 
p = pyaudio.PyAudio()

 
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)
 
x = np.arange(0, 2 * CHUNK, 2)      
xf = np.linspace(0, RATE, CHUNK)    

 
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)

 
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

 
ax1.set_title('Графік зміни тиску від часу')
ax1.set_xlabel('фрагмент')
ax1.set_ylabel('значення')
ax1.set_ylim(0, 255)
ax1.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])

 
ax2.set_xlim(20, RATE / 2)

print('розпочався потік')

 
frame_count = 0
start_time = time.time()

while True:
    data = stream.read(CHUNK)  
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    data_np = np.array(data_int, dtype='b')[::2] + 128
    line.set_ydata(data_np)
    yf = fft(data_int)
    line_fft.set_ydata(np.abs(yf[0:CHUNK])  / (128 * CHUNK))
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
       
    except TclError:

        frame_rate = frame_count / (time.time() - start_time)
       
        print('потік зупинився')
        print('середня частота кадрів = {:.0f} FPS'.format(frame_rate))
        break


розпочався потік
